In [83]:
# Import required libraries
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output, State
from jupyter_dash import JupyterDash
import plotly.graph_objects as go
import plotly.express as px
from dash import no_update


# Create a dash application
app = JupyterDash(__name__)
JupyterDash.infer_jupyter_proxy_config()

# REVIEW1: Clear the layout and do not display exception till callback gets executed
app.config.suppress_callback_exceptions = True

# Read the airline data into pandas dataframe
spacex_launch_data =  pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv", 
                            encoding = "ISO-8859-1",
                            dtype={'Div1Airport': str, 'Div1TailNum': str, 
                                   'Div2Airport': str, 'Div2TailNum': str})

LaunchSite_list=spacex_launch_data['Launch Site'].value_counts()
LaunchSite_list=LaunchSite_list.index.tolist()
labellist=[{'label':"ALL", 'value': "ALL"}]+[{'label': i, 'value': i} for i in LaunchSite_list]

max_payload = spacex_launch_data['Payload Mass (kg)'].max()
min_payload = spacex_launch_data['Payload Mass (kg)'].min()


# List of years 
year_list = [i for i in range(2005, 2021, 1)]
# Application layout
# Application layout
app.layout = html.Div(children=[ 
                                # TODO1: Add title to the dashboard
                                html.H1('SpaceX Launch Records Dashboard', style={'textAlign':'center', 'color':'#503D36', 'font-size':24}),

                                # REVIEW2: Dropdown creation
                                # Create an outer division 
#                                 html.Div([
                                    # Add an division
                                    html.Div([
                                        # Create an division for adding dropdown helper text for report type
                                        html.Div(
                                            [
                                            html.H2('Launch Type:', style={'margin-right': '2em'}),
                                            ]
                                        ),
                                        # TODO2: Add a dropdown
                                        dcc.Dropdown(id='site_dropdown', 
                                          options=labellist,
                                          
                                          placeholder="place holder here",
                                          searchable=True,
                                          style={'width':'80%', 'padding':'3px', 'font-size': '20px', 'text-align-last' : 'center'}),
                                    # Place them next to each other using the division style
                                    ], style={'display':'flex'}),
#                                     ,

#                                    # Add next division 
#                                    html.Div([
#                                        # Create an division for adding dropdown helper text for choosing year
#                                         html.Div(
#                                             [
#                                             html.H2('Choose Year:', style={'margin-right': '2em'})
#                                             ]
#                                         ),
#                                         dcc.Dropdown(id='input-year', 
#                                             # Update dropdown values using list comphrehension
#                                             options=[{'label': i, 'value': i} for i in year_list],
#                                             placeholder="Select a year",
#                                             style={'width':'80%', 'padding':'3px', 'font-size': '20px', 'text-align-last' : 'center'}),
#                                             # Place them next to each other using the division style
#                                             ], style={'display': 'flex'}),  
#                                         ]),
                                
                                # Add Computed graphs
                                # REVIEW3: Observe how we add an empty division and providing an id that will be updated during callback
                                    html.Div(children=[html.Div([],id="success-pie-chart")], style={'display': 'flex'})
                            
#                                 ,
    
#                                 html.Div(children=[
#                                         html.Div([ ], id='plot2'),
#                                         html.Div([ ], id='plot3')
#                                 ], style={'display': 'flex'}),
                                
#                                 # TODO3: Add a division with two empty divisions inside. See above disvision for example.
#                                 html.Div(children=[
#                                         html.Div([ ], id='plot4'),
#                                         html.Div([ ], id='plot5')
#                                 ], style={'display': 'flex'}),
                                    ,                                
                                    dcc.RangeSlider(
                                        id='payload_slider',
                                        min=0,
                                        max=10000,
                                        step=1000,  
                                        marks = {
                                                0: '0 \nkg',
                                                1000: '1000 \nkg',
                                                2000: '2000 \nkg',
                                                3000: '3000 \nkg',
                                                4000: '4000 \nkg',
                                                5000: '5000 \nkg',
                                                6000: '6000 \nkg',  
                                                7000: '7000 \nkg',
                                                8000: '8000 \nkg',
                                                9000: '9000 \nkg',
                                                10000: '10000 \nkg'                                         
                                        },

                                        value=[min_payload,max_payload]
                                    ),
                                html.Div(dcc.Graph(id='success-payload-scatter-chart'))

                                 ])


In [84]:
@app.callback(Output(component_id='success-pie-chart', component_property='children'),
              [Input(component_id='site_dropdown', component_property='value')]
#               ,              [State("success-pie-chart", 'children')]
             )
def get_pie_chart(site_dropdown):
#     filtered_df = spacex_df
    if site_dropdown == 'ALL':
        fig = px.pie(spacex_launch_data.loc[spacex_launch_data['class'] == 1], values='class', 
        names='Launch Site', 
        title='Total Success Launches for all type') # ,hole=.3 中空程度
        return ([dcc.Graph(figure=fig)])
    else:
        if site_dropdown:
            fig = px.pie(spacex_launch_data.loc[spacex_launch_data['Launch Site']==site_dropdown],  names='class', 
    #         names='Launch Site', 
            title='Total Success Launches for type'+site_dropdown)
            return ([dcc.Graph(figure=fig)])
    
@app.callback(
     Output(component_id='success-payload-scatter-chart',component_property='figure'),
     [Input(component_id='site_dropdown',component_property='value'),Input(component_id="payload_slider", component_property="value")]               
)
def update_scattergraph(site_dropdown,payload_slider):
    if site_dropdown == 'ALL':
        low, high = payload_slider
        df  = spacex_launch_data
        mask = (df['Payload Mass (kg)'] > low) & (df['Payload Mass (kg)'] < high)
        fig = px.scatter(
            df[mask], x="Payload Mass (kg)", y="class", 
            color="Booster Version",
            size='Payload Mass (kg)',
            hover_data=['Payload Mass (kg)'],
            title='Weighted Payload range from'+' '+str(low)+' '+'to'+' '+str(high))
    else:
        low, high = payload_slider
        df  = spacex_launch_data.loc[spacex_launch_data['Launch Site'] == site_dropdown]
        mask = (df['Payload Mass (kg)'] > low) & (df['Payload Mass (kg)'] < high)
        fig = px.scatter(
            df[mask], x="Payload Mass (kg)", y="class", 
            color="Booster Version",
            size='Payload Mass (kg)',
            hover_data=['Payload Mass (kg)'],
            title='Weighted Payload range from'+' '+str(low)+' '+'to'+' '+str(high))
    return fig

In [85]:
# Run the app
if __name__ == '__main__':
    # REVIEW8: Adding dev_tools_ui=False, dev_tools_props_check=False can prevent error appearing before calling callback function
    app.run_server(mode="inline", host="localhost", debug=False, dev_tools_ui=False, dev_tools_props_check=False)

127.0.0.1 - - [03/Jan/2022 04:46:00] "GET /_shutdown_ac115c26-1cfe-4b04-8dc1-b31d1ebd89a1 HTTP/1.1" 200 -
 * Running on http://localhost:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [03/Jan/2022 04:46:02] "GET /_alive_ac115c26-1cfe-4b04-8dc1-b31d1ebd89a1 HTTP/1.1" 200 -


127.0.0.1 - - [03/Jan/2022 04:46:03] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Jan/2022 04:46:03] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [03/Jan/2022 04:46:03] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [03/Jan/2022 04:46:03] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Jan/2022 04:46:04] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/Jan/2022 04:46:04] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/Jan/2022 04:46:04] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Jan/2022 04:46:04] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Jan/2022 04:46:04] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Jan/2022 04:46:14] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/Jan/2022 04:46:14] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/Jan/2022 